In [5]:
import pandas as pd
import re

text_df = pd.read_csv('gutenberg/gutenberg.csv')

text_df.head()

,ID,author,title,subtitle,vers
0,0,alexis,Entführung,NaN,"O Lady Judith, spröder Schatz,\n Drückt..."
1,0,alexis,Entführung,NaN,"Hart ist der Sitz und knapp und schmal,\n ..."
2,0,alexis,Entführung,NaN,Sechs Nächte lag ich in Sumpf und Moor\n ...
3,1,alexis,Walpurgisnacht,NaN,"Liebe Mutter, heut' Nacht heulte Regen und Win..."
4,1,alexis,Walpurgisnacht,NaN,"Liebe Mutter, es donnerte auf dem Brocken drob..."


In [2]:
verse_name = 'vers'

# Preprocessing for Bert Training

In [5]:
# EOV: End of verse; SOV: Start of Verse; SST: Start of Strophe; EST: End of Strophe
# SOP: Start of poem; EOP: End of Poem

text_df_prep = text_df.copy()
text_df_prep[verse_name] = text_df_prep[verse_name].apply(lambda x: re.sub(r'[^a-zäöüA-ZÄÖÜ,.\n]',' ', str(x)))
text_df_prep[verse_name] = text_df_prep[verse_name].apply(lambda x: (' [EOV] [SOV] ').join([line.lstrip() for line in str(x).split('\n') if line] )+ ' [EST] ')

#text_df_prep = text_df_prep.groupby(['title']).sum()



text = ''
for _, row in text_df_prep.iterrows():
    
    text += row[verse_name]
    
with open('bert_training_gutenberg.txt', 'w') as f:
    f.write(text)

# Preprocessing for GPT2 Training

In [2]:
text_df = pd.read_csv('gutenberg/gutenberg.csv')

text_df.head()

,ID,author,title,subtitle,vers
0,0,alexis,Entführung,NaN,"O Lady Judith, spröder Schatz,\n Drückt..."
1,0,alexis,Entführung,NaN,"Hart ist der Sitz und knapp und schmal,\n ..."
2,0,alexis,Entführung,NaN,Sechs Nächte lag ich in Sumpf und Moor\n ...
3,1,alexis,Walpurgisnacht,NaN,"Liebe Mutter, heut' Nacht heulte Regen und Win..."
4,1,alexis,Walpurgisnacht,NaN,"Liebe Mutter, es donnerte auf dem Brocken drob..."


In [7]:
title_name = 'title'
verse_name = 'vers'


text_df_prep = text_df.copy()
text_df_prep[verse_name] = text_df_prep[verse_name].apply(lambda x: re.sub(r'[^a-zäöüA-ZÄÖÜß?!,;:.\n ]','', str(x)))
text_df_prep[verse_name] = text_df_prep[verse_name].apply(lambda x: ('\n').join([line.lstrip() for line in str(x).split('\n') if line] )+ '\n')

text_df_strophes = text_df_prep.groupby(['ID']).sum()[verse_name]

text_df_titles = text_df_prep.groupby(['ID']).first()[title_name]

strophes_df = pd.concat([text_df_titles, text_df_strophes], axis=1)

text = ''
for _ , row in strophes_df.iterrows():
    
    if 'h4' in row[title_name]:
        title = 'Gedicht'
    else: 
        title = row[title_name]
    text += 'Titel: ' + str(title) + '\n \n' + row[verse_name] + '<|endoftext|>'+'\n'
    
    
    
with open('gpt2_training_gutenberg.txt', 'w') as f:
    f.write(text)

In [11]:
re.sub('ch', '2', 'ichde')

'i2de'